## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [1]:
!git clone https://github.com/Yui-Arthur/WMamba.git > /dev/null
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git > /dev/null

Cloning into 'WMamba'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 158 (delta 46), reused 39 (delta 19), pack-reused 68
Receiving objects: 100% (158/158), 102.87 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/kaggle/working/WMamba
Cloning into 'U-Mamba'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 301 (delta 46), reused 18 (delta 16), pack-reused 231
Receiving objects: 100% (301/301), 2.91 MiB | 14.99 MiB/s, done.
Resolving deltas: 100% (74/74), done.


#### Mabma & U-Mamba & WMamba reqirement

In [2]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [3]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [4]:
!sudo bash setup_wmamba.sh

Setup WMamba
CP File From W-Mamba to U-Mamba
Trainer:
  nnUNetTrainerWMambaBase.py
  nnUNetTrainerUMambaEncNoAMP.py
  nnUNetTrainerUMambaEnc.py
  nnUNetTrainerWMambaBot.py
  nnUNetTrainerUMambaBot.py
Nets:
  WMambaBot_3d.py
Others:
  paths.py
  waveletLayer.py


### set env variable

In [5]:
### DO NOT EDIT !!! ###
nnUNet_preprocessed="/home/nnUNet_preprocessed/"
nnUNet_raw="/home/nnUNet_raw/"
nnUNet_results="/kaggle/working/"

%env nnUNet_preprocessed={nnUNet_preprocessed}
%env nnUNet_raw={nnUNet_raw}
%env nnUNet_results={nnUNet_results}



env: nnUNet_preprocessed=/home/nnUNet_preprocessed/
env: nnUNet_raw=/home/nnUNet_raw/
env: nnUNet_results=/kaggle/working/


### parameter (can modify)

In [6]:
### Epoch ###
# **the maximum epoch include all train epoch in every session** #
%env training_epochs=150

### Checkpoint ###
# every {checkpoint_save_every} epochs will save one checkpoint
%env checkpoint_save_every = 10
load_checkpoint = True
prev_checkpoint_f0 = "/kaggle/input/0505_1/checkpoint_latest_0.pth"
prev_checkpoint_f1 = "/kaggle/input/0505_1/checkpoint_latest_1.pth"

### Kaggle Dataset Input Path ###
dataset_path = "/kaggle/input/Dataset110_CTImage.mp4"
### Dataset ID ###
dataset_id = 110

### Set Trainer/Model ###
trainer = "nnUNetTrainerWMambaBot"

### Set GPU DRAM for dataset preproccess ###
gpu_dram = 8


env: training_epochs=150
env: checkpoint_save_every=10


### dataset prepaired

In [7]:
if dataset_id == 110:
    """===== Dataset 110 Covid-19 ===="""
    """ unrar data folder and move to home. """
    !apt install unrar
    !(unrar x {dataset_path} {nnUNet_raw}) >  /dev/null
    """dataset preoproccess about 20 mins."""
    !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 110
    dataset_name = "Dataset110_CTImage"
else:
    """===== Dastaset 4 Hippocampus ===="""
    """ unzip data folder and move to home. """
    !(tar -C  /home/ -xvf {dataset_path}) > /dev/null
    """ dataset preoproccess. """
    !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus 
    !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 4
    dataset_name = "Dataset004_Hippocampus"





The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 51 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
Fetched 113 kB in 0s (511 kB/s)

78Selecting previously unselected package unrar.
(Reading database ... 113807 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.6.6-2build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unrar (1:5.6.6-2build1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up unrar (1:5.6.6-2build1) ...
7Progress: [ 60%] [##################################........................] 8update-alternatives: using /usr/bin/unrar-nonfree to pro

### train

In [8]:
from pathlib import Path
if not load_checkpoint:
    ### first training (no checkpoint) ###
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --npz
else:
    ### load checkpoint, continue train
    current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"

    !mkdir -p {current_trainer_folder}/fold_0
    !mkdir -p {current_trainer_folder}/fold_1

    !cp {prev_checkpoint_f0} {current_trainer_folder}/fold_0/checkpoint_best.pth
    !cp {prev_checkpoint_f1} {current_trainer_folder}/fold_1/checkpoint_best.pth
    
    # check the checkpoint can be successful loaded
    f0 = [i.name for i in Path(current_trainer_folder+"fold_0").glob("*.pth")]
    f1 = [i.name for i in Path(current_trainer_folder+"fold_1").glob("*.pth")]
    print("Load Checkpoint fold 0",f0)
    print("Load Checkpoint fold 1",f1)
    if len(f0) <1 or len(f1) <1:
        raise ValueError("Load Checkpoint Failed")
    
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --c --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --c  --npz

Load Checkpoint fold 0 ['checkpoint_best.pth']
Load Checkpoint fold 1 ['checkpoint_best.pth']
2024-05-05 08:24:01.412178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 08:24:01.412170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 08:24:01.412251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 08:24:01.412283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 08:24:01.580853: E external/local_xla/xla/stream_exec

### find best fold

In [9]:
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1


***All results:***
nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres: 0.705625379485016

*Best*: nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres: 0.705625379485016

***Determining postprocessing for best model/ensemble***
Removing all but the largest foreground region did not improve results!

***Run inference like this:***

nnUNetv2_predict -d Dataset110_CTImage -i INPUT_FOLDER -o OUTPUT_FOLDER -f  0 1 -tr nnUNetTrainerWMambaBot -c 3d_fullres -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file /kaggle/working/Dataset110_CTImage/nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres/crossval_results_folds_0_1/postprocessing.pkl -np 8 -plans_json /kaggle/working/Dataset110_CTImage/nnUNetTrainerWMambaBot__nnUNetPlans__3d_fullres/crossval_results_folds_0_1/plans.json


### predict 

In [10]:
!nnUNetv2_predict -i "{nnUNet_raw}{dataset_name}/imagesTs"\
-o "{nnUNet_results}{dataset_name}/predTs" \
-d {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 > /dev/null

2024-05-05 17:05:22.859712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 17:05:22.859857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 17:05:22.978549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|███████████████████████████████████████████| 27/27 [00:56<00:00,  2.11s/it]


In [11]:
!python ./createSubmissionFile.py -s "{nnUNet_results}{dataset_name}/predTs" -d "{nnUNet_results}/predTs_rename"
!tar zcvf pred.tar.gz "{nnUNet_results}/predTs_rename"

[01/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0318.nii.gz to /kaggle/working//predTs_rename/318_0.nii.gz
[02/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0302.nii.gz to /kaggle/working//predTs_rename/302.nii.gz
[03/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0543.nii.gz to /kaggle/working//predTs_rename/543.nii.gz
[04/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0268.nii.gz to /kaggle/working//predTs_rename/268.nii.gz
[05/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0451.nii.gz to /kaggle/working//predTs_rename/451.nii.gz
[06/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0369.nii.gz to /kaggle/working//predTs_rename/369_1.nii.gz
[07/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0516.nii.gz to /kaggle/working//predTs_rename/516.nii.gz
[08/50] Convert file from /kaggle/working/Dataset110_CTImage/pred